<a href="https://colab.research.google.com/github/mana5aS/Visual-Odometry-using-RGB-D-Images/blob/main/Visual_Odometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Odometry using RGB-D Images.

In [ ]:
#imports 

from scipy.spatial import kdtree
import numpy as np
import copy
import cv2 as cv
import matplotlib.pyplot as plt
from matplotlib import image
import time
import math
from tqdm import tqdm
from PIL import Image
from collections import Counter

In [ ]:
#opencv version to use SIFT 
!pip install opencv-contrib-python==4.5.5.62 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Add path to the dataset in the drive
path = ''

Mounted at /content/drive


### Dataset Processing 

In [ ]:
def dataset_processing():
    """
    Function to perform association of RGB Images and Depth Images based on timestamp

    Returns
    -------
        depth_time - timestamps of depth images
        depth_file - filenames of depth images
        rgb_time   - timestamps of rgb images
        rgb_file   - filenames of rgb images
        gt_time    - timestamps of ground truth
        t_gt       - ground truth translations
        q_gt       - ground truth quaternions (orientation)

    """

    #Parsing Depth Images
    file1 = open(path + "depth.txt","r+") 
    file1.seek(75)

    depth_time = []
    depth_file = []
    flag = False
    for line in file1:
        if flag:
            depth_time.append(float(line.split()[0]))
            depth_file.append(line.split()[1])
        flag = True

    depth_time = np.asarray(depth_time,dtype=np.float128)
    depth_file = np.asarray(depth_file)


    #Parsing RGB Images 
    file2 = open(path + "rgb.txt","r+") 
    file2.seek(77)
    rgb_time = []
    rgb_file = []
    flag = False
    for line in file2:
        if flag:
            rgb_time.append(float(line.split()[0]))
            rgb_file.append(line.split()[1])
        flag = True


    rgb_time = np.asarray(rgb_time,dtype=np.float128)
    rgb_file = np.asarray(rgb_file)

    #Parsing Ground Truth 
    file3 = open(path + "groundtruth.txt","r+") 
    file3.seek(101)

    gt_time = []
    gt_tx = []
    gt_ty = []
    gt_tz = []
    gt_quatx = []
    gt_quaty = []
    gt_quatz = []
    gt_quatw = []

    for line in file3:
        data = line.split()
        gt_time.append(float(data[0]))
        gt_tx.append(float(data[1]))
        gt_ty.append(float(data[2]))
        gt_tz.append(float(data[3]))
        gt_quatx.append(float(data[4]))
        gt_quaty.append(float(data[5]))
        gt_quatz.append(float(data[6]))
        gt_quatw.append(float(data[7]))

    gt_time = np.asarray(gt_time,dtype=np.float128)
    gt_tx = np.asarray(gt_tx)
    gt_ty = np.asarray(gt_ty)
    gt_tz = np.asarray(gt_tz)
    t_gt = np.stack((gt_tx, gt_ty, gt_tz))
    gt_quatx = np.asarray(gt_quatx)
    gt_quaty = np.asarray(gt_quaty)
    gt_quatz = np.asarray(gt_quatz)
    gt_quatw = np.asarray(gt_quatw)
    q_gt = np.stack((gt_quatx, gt_quaty, gt_quatz, gt_quatw))


    return depth_time, depth_file, rgb_time, rgb_file, gt_time, t_gt, q_gt


In [ ]:
""" Function to find matching RGB Images and Ground Truth values for each Depth Image - based on timestamp """
img_idx = lambda t: np.argmin(np.abs(rgb_time - t))
img_idx2 = lambda t: np.argmin(np.abs(gt_time - t))

### 2D Feature Extraction and 3D Distribution Generation (with modeled depth uncertainty) 

In [ ]:
def get_z_mean_var(u, v, depth, size=3):
    """
    Function that uses Gaussian Mixture Models to model the uncertainty in depth (z) and
    return mean and variance.

    Parameters
    ----------
        u, v  - image pixels
        depth - (H,W) - depth image 
        size  - window size

    Returns
    -------
        z_mean - mean of depth 
        sz_sq  - variance of depth

    """

    G_kernel = np.array([[1, 2 , 1],[2, 4, 2], [1, 2, 1]]) * (1/16)  # gaussian kernel
    factor = 5000
    
    depth_padded = np.pad(depth, 1, mode='constant')
    
    # window  
    rs = (v + 1) - size//2
    re = (v + 1) + size//2 + 1
    cs = (u + 1) - size//2
    ce = (u + 1) + size//2 + 1

    # filtered depth values
    depth_window = depth_padded[rs:re, cs:ce] / factor

    # z mean
    weighted_depth = G_kernel * depth_window
    z_mean = np.sum(weighted_depth)

    # z variance
    depth_sq_window = np.square(depth_window)
    std_window = (0.001425) * depth_sq_window 
    var_window = np.square(std_window)
    temp = G_kernel * (var_window + depth_sq_window) 
    sz_sq = np.sum(temp) - (z_mean**2)  

    return z_mean, sz_sq

In [ ]:
def cov_GMM(u, v, z, sz_sq, fx, fy, cx, cy, depth):
    """
    Function that builds the covariance matrix for each feature using the 
    newly calculated z mean and variance (after noise modeling)

    Parameters
    ----------
        u, v   - image pixels
        z      - z mean
        sz_sq  - z variance
        fx, fy - focal lengths
        cx, cy - optical centers
        depth  - (H,W) - depth image 

    Returns
    -------
        D_cov  - Covariance Matrix 
        
    """

    su = 1.0    # standard deviation of u pixel coordinate
    sv = 1.0    # standard deviation of v pixel coordinate

    sx_sq = (sz_sq*(u-cx)*(v-cy) + (su**2 * (z**2 + sz_sq))) / fx**2
    sy_sq = (sz_sq*(u-cx)*(v-cy) + (sv**2 * (z**2 + sz_sq))) / fy**2
    sxz = sz_sq*(u-cx) / fx
    syz = sz_sq*(v-cy) / fy
    sxy = (sz_sq*(u-cx)*(v-cy)) / (fx*fy)
    szx = sxz
    szy = syz
    syx = sxy
    D_cov = np.array([[sx_sq, sxy, sxz],
                    [sxy, sy_sq, syz],
                    [sxz, szy, sz_sq]])
      
    return D_cov



In [ ]:
def get_data_gftt(image, depth):
    """
    Function that extracts Shi-Tomasi Features and builds feature set D.

    Parameters
    ----------
        image - (H,W) - RGB Image 
        depth - (H,W) - Depth Image 

    Returns
    -------
        D_mean - (N,4) - Mean of each feature   
        D_cov  - (N,3,3) - Covariance Matrix of each feature  
        
    """  

    #### Feature Extraction - Shi-Tomasi Features   
    image = np.asarray(image)
    depth = np.asarray(depth)
    gray = cv.cvtColor(image , cv.COLOR_BGR2GRAY)
    corners = cv.goodFeaturesToTrack(gray,400,0.01,2)
    corners = np.int0(corners)

    fx = 525.0  # focal length x
    fy = 525.0  # focal length y
    cx = 319.5  # optical center x
    cy = 239.5  # optical center y

    factor = 5000 # for the 16-bit PNG files

    #### Calculate D_mean and D_cov
    D_mean, D_cov = [], []

    max_z_mean = 5.5
    max_z_var = 0.03
    for i in corners:
        u, v = i.ravel()
        z = depth[v, u] / factor    # raw depth value
        if z == 0:                  # missing/invalid depth values (according to dataset)
            continue
        
        # get mean and variance of raw depth value
        z_mean, sz_sq = get_z_mean_var(u, v, depth)
        if((z_mean > max_z_mean) or (sz_sq > max_z_var)):  # removing outliers
            continue

        z = z_mean
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy

        D_mean.append([x,y,z,1])    
        D_cov.append(cov_GMM(u, v, z, sz_sq, fx, fy, cx, cy, depth))
  

    D_mean = np.asarray(D_mean)
    D_cov = np.asarray(D_cov)

    return D_mean, D_cov

  

In [ ]:
def get_data_sift(image, depth): 
    """
    Function that extracts SIFT Features and builds feature set D.

    Parameters
    ----------
        image - (H,W) - RGB Image 
        depth - (H,W) - Depth Image 

    Returns
    -------
        D_mean - (N,4) - Mean of each feature   
        D_cov  - (N,3,3) - Covariance Matrix of each feature  
        
    """   

    #### Feature Extraction - SIFT Features
    image = np.asarray(image)
    depth = np.asarray(depth)
    gray = cv.cvtColor(image , cv.COLOR_BGR2GRAY)

    sift = cv.xfeatures2d.SIFT_create(400)
    kp = sift.detect(gray, None)
    pts = cv.KeyPoint_convert(kp)
    pts = np.round(pts).astype(int)

    fx = 525.0  # focal length x
    fy = 525.0  # focal length y
    cx = 319.5  # optical center x
    cy = 239.5  # optical center y

    factor = 5000 # for the 16-bit PNG files

    #### Calculate D_mean and D_cov
    D_mean, D_cov = [], []

    max_z_mean = 5.5
    max_z_var = 0.03
    for pt in pts:
        u = pt[0]
        v = pt[1]
        z = depth[v, u] / factor  
        if z == 0:
            continue
        
        z_mean, sz_sq = get_z_mean_var(u, v, depth)
        if((z_mean > max_z_mean) or (sz_sq > max_z_var)):
            continue

        z = z_mean
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy

        D_mean.append([x,y,z,1])    
        D_cov.append(cov_GMM(u, v, z, sz_sq, fx, fy, cx, cy, depth))

    D_mean = np.asarray(D_mean)
    D_cov = np.asarray(D_cov)

    return D_mean, D_cov

In [ ]:
def get_data_orb(image, depth): 
    """
    Function that extracts ORB Features and builds feature set D.

    Parameters
    ----------
        image - (H,W) - RGB Image 
        depth - (H,W) - Depth Image 

    Returns
    -------
        D_mean - (N,4) - Mean of each feature   
        D_cov  - (N,3,3) - Covariance Matrix of each feature  
        
    """  

    #### Feature Extraction - ORB Features  
    image = np.asarray(image)
    depth = np.asarray(depth)
    gray = cv.cvtColor(image , cv.COLOR_BGR2GRAY)

    orb = cv.ORB_create(400)
    kp = orb.detect(gray, None)
    pts = cv.KeyPoint_convert(kp)
    pts = np.round(pts).astype(int)

    fx = 525.0  # focal length x
    fy = 525.0  # focal length y
    cx = 319.5  # optical center x
    cy = 239.5  # optical center y

    factor = 5000 # for the 16-bit PNG files

    #### Calculate D_mean and D_cov
    D_mean, D_cov = [], []

    max_z_mean = 5.5
    max_z_var = 0.03
    for pt in pts:
        u = pt[0]
        v = pt[1]
        z = depth[v, u] / factor   
        if z == 0:
            continue
        
        z_mean, sz_sq = get_z_mean_var(u, v, depth)
        if((z_mean > max_z_mean) or (sz_sq > max_z_var)):
            continue

        z = z_mean
        x = (u - cx) * z / fx
        y = (v - cy) * z / fy

        D_mean.append([x,y,z,1])    
        D_cov.append(cov_GMM(u, v, z, sz_sq, fx, fy, cx, cy, depth))

    D_mean = np.asarray(D_mean)
    D_cov = np.asarray(D_cov)

    return D_mean, D_cov

### Registration using Iterative Closest Point 

In [ ]:
def get_k_nearest_neigh(M_mean, D_mean, k):
    """
    Function that uses KD-Tree to find the k nearest neighbors in M for every 
    feature in D

    Parameters
    ----------
        M_mean - (N,4) - mean of model (M) features
        D_mean - (N,4) - mean of data (D) features
        k              - no. of neighbors

    Returns
    -------
        dists - (N,k)  - Euclidean distances between each D features and its 
                         'k' neighbors
        neigh_idxs - (N,) - indices of the neighbors in M

    """

    tree = kdtree.KDTree(M_mean)
    dists, neigh_idxs = tree.query(D_mean, k) 
    
    return dists, neigh_idxs 


In [ ]:
def find_nearest_neigh_icp(M_mean, M_cov, D_mean, D_cov, euc_dist_th = 0.15):
    """
    Function that finds the nearest neigbors for Registration (using ICP)

    Parameters
    ----------
        M_mean  - (N,4)    - mean of model (M) features
        M_cov   - (N,3,3)  - covariance of model (M) features
        D_mean  - (N,4)    - mean of data (D') features
        D_cov   - (N,3,3)  - covariance of data (D') features
        euc_dist_th        - threshold for registration
       
    Returns
    -------
        d_associated_idxs - indices in D that are associated for registration
        m_associated_idxs - indices in M that are associated for registration

    """

    dists, neigh_idxs = get_k_nearest_neigh(M_mean, D_mean, k=1)

    min_dist = np.inf
    min_dist_idx = 0
    d_associated_idxs = []
    m_associated_idxs = []
    nearest_dist = []

    for i in range(D_mean.shape[0]):    
        if dists[i] < euc_dist_th:
            d_associated_idxs.append(i)
            m_associated_idxs.append(neigh_idxs[i])
            nearest_dist.append(dists[i])
            
    return d_associated_idxs, m_associated_idxs
        

In [ ]:
def find_T(X, Y):
    """
    Function that computes the transformation between two point clouds (Procrustes)
    Y = RX + t

    Parameters
    ----------
        X, Y - (N,3), (N,3) - Two point clouds

    Returns
    -------
        T - (4,4) - Transformation matrix

    """

    X_centroid = np.mean(X.T,axis = 1).reshape(3,1)
    Y_centroid = np.mean(Y.T,axis = 1).reshape(3,1)
    A = X.T - X_centroid 
    B = Y.T - Y_centroid 

    u,s,vT = np.linalg.svd(A @ B.T)

    S = np.eye(3)
    S[2][2] = np.linalg.det(vT.T@u.T)
    R = vT.T @ S @ u.T
    t = (Y_centroid - (R @ X_centroid)).reshape(-1)

    T = np.eye(4)
    T[:3,:3] = copy.deepcopy(R)
    T[:3,3] = copy.deepcopy(t)
    
    return T

In [ ]:
def icp(D_mean, D_cov, M_mean, M_cov, T_prev, max_iter=10, linear_tol=1e-4, angular_tol=1.7e-3, min_corres = 15):
    """
    Function that performs registration: finds the best estimate of the transformation matrix
    between Data feature set (D) and Model feature set (M)

    Parameters
    ----------
        D_mean - (N,4)   - mean of data (D) features
        D_cov  - (N,3,3) - covariance of data (D) features
        M_mean - (N,4)   - mean of model (M) features
        M_cov  - (N,3,3) - covariance of model (M) features
        T_prev - (4,4)   - previous estimate of T
       
    Returns
    -------
        T_final - (4,4) - best estimate of transformation matrix

    """    

    # Transform dataset D_mean to fixed frame using preious transformation estimate
    D_mean_transformed = (T_prev @ D_mean.T).T
    R = T_prev[:3,:3]
    D_cov_transformed = R @ D_cov @ R.T

    T_final = np.eye(4)

    for i in range(max_iter):
        d_assoc_idxs, m_assoc_idxs = find_nearest_neigh_icp(M_mean, M_cov, D_mean_transformed, D_cov_transformed)

        if len(m_assoc_idxs) < min_corres:
            break

        # find new transformation matrix
        T_corr = find_T(D_mean_transformed[d_assoc_idxs, :3], M_mean[m_assoc_idxs,:3])

        # update D_mean
        D_mean_transformed = (T_corr @ D_mean_transformed.T).T
        # accumulate Final transformation matrix
        T_final = T_corr @ T_final
        
        # check for convergence
        t = T_corr[:3,3]
        R = T_corr[:3,:3]

        # linear error
        linear_err = np.linalg.norm(t)
        tr = R[0,0] + R[1,1] + R[2,2]

        # angular_err 
        cos_theta = (tr - 1.0)/2.0
        cos_theta = np.clip(cos_theta,-1,1)
        angular_err = np.arccos(cos_theta)
        if((linear_err <linear_tol) and (angular_err < angular_tol)):
            break

    return T_final

### Data Association 

In [ ]:
def mahalanobis_dist(d_mean, d_cov, m_mean, m_cov):
    """
    Function that calculates the Mahalanobis distance between Data feature set D
    and Model feature set M

    Parameters
    ----------
        d_mean - (N,4) - mean of data (D) features 
        d_cov  - (N,3,3) - covariance of data (D) features
        m_mean - (N,4) - mean of model (M) features
        m_cov  - (N,3,3) - covariance of model (M) features

    Returns
    -------
        d - (1,N) - Mahalanobis distances 

    """
    
    delta = m_mean - d_mean
    delta = delta[:,:3]
    d_2 = delta[:,np.newaxis,:] @ np.linalg.inv(m_cov + d_cov) @ delta[:,:,np.newaxis]
    d = np.sqrt(d_2)

    return d.reshape(1,-1)

In [ ]:
def find_nearest_neigh_assoc(M_mean, M_cov, D_mean, D_cov, maha_dist_th):
    """
    Function that finds the nearest neigbors for Data Association

    Parameters
    ----------
        M_mean - (N,4)   - mean of model (M) features
        M_cov  - (N,3,3) - covariance of model (M) features
        D_mean - (N,4)   - mean of data (D') features
        D_cov  - (N,3,3) - covariance of data (D') features
        maha_dist_th     - threshold for association
       
    Returns
    -------
        d_associated_idxs - indices in Dp that are associated
        m_associated_idxs - indices in M that are associated
        nearest_dist      - nearest distances

    """    

    d_associated_idxs = []
    m_associated_idxs = []
    nearest_dist = []

    neigh_idxs = get_k_nearest_neigh(M_mean, D_mean, k=4)[1]

    for i in range(D_mean.shape[0]):
        dist = mahalanobis_dist(D_mean[i], D_cov[i], M_mean[neigh_idxs[i]], M_cov[neigh_idxs[i]])
        if np.isnan(dist).all():
            continue        
        else:
            min_dist = np.nanmin(dist)
            nearest_dist.append(min_dist)
            if min_dist < maha_dist_th:
                d_associated_idxs.append(i)
                m_associated_idxs.append(neigh_idxs[i][np.nanargmin(dist)])
            
    return d_associated_idxs, m_associated_idxs, nearest_dist
        

In [ ]:
def association_idx(Dp_mean, Dp_cov, M_mean, M_cov, th = 11.35):
    """
    Function that returns the indices of associated features in Dp and M

    Parameters
    ----------
        Dp_mean - (N,4)     - mean of data (D') features
        Dp_cov  - (N,3,3)   - covariance of data (D') features
        M_mean  - (N,4)     - mean of model (M) features
        M_cov   - (N,3,3)   - covariance of model (M) features
        th                  - threshold for association
       
    Returns
    -------
        d_assoc_idxs - indices in Dp that are associated
        m_assoc_idxs - indices in M that are associated
        dist_arr     - all distances (for plotting)

    """   
     
    # obtaining associated indices
    d_assoc_idxs, m_assoc_idxs, dist_arr = find_nearest_neigh_assoc(M_mean, M_cov, Dp_mean, Dp_cov, th)

    dist_arr = np.asarray(dist_arr)
    dist_arr = np.squeeze(dist_arr)
    dist_arr = dist_arr.flatten()


    return d_assoc_idxs, m_assoc_idxs, dist_arr          

In [ ]:
def Kalman_Update(D_assoc_mean, D_assoc_cov, M_assoc_mean, M_assoc_cov): 
    """
    Function that updates associated features in Model M using Kalman Filter

    Parameters
    ----------
        D_assoc_mean  - (N,4)       - mean of associated data (D') features
        D_assoc_cov   - (N,3,3)     - covariance of associated data (D') features
        M_assoc_mean  - (N,4)       - mean of associated model (M) features
        M_assoc_cov   - (N,3,3)     - covariance of associated model (M) features
       
    Returns
    -------
        M_upd_mean  - (N,4)     - Updated Mean value of Model (M)
        M_upd_cov   - (N,3,3)   - Updated Covariance value of Model (M)

    """   

    #propagate dynamics
    inter_mean = copy.deepcopy(M_assoc_mean)   #(N,4)
    inter_cov = copy.deepcopy(M_assoc_cov)  #(N,3,3)

    #updating using observations
    K = inter_cov @ np.linalg.inv(inter_cov + D_assoc_cov)
    
    M_upd_mean = (inter_mean[:, :3, np.newaxis] + (K @ np.subtract(D_assoc_mean, inter_mean)[:, :3, np.newaxis])).reshape(-1,3)
    I = np.eye(3)
    M_upd_cov = np.subtract(I, K) @ inter_cov
    
    M_upd_mean = np.hstack((np.array(M_upd_mean), np.ones((len(M_upd_mean), 1))))
    M_upd_cov = np.array(M_upd_cov)


    return M_upd_mean, M_upd_cov    #(N,4), (N,3,3)

### Main function

In [ ]:
""" Main Function """

#Select feature detector
feature_detector = "GFTT"    # "GFTT" or "SIFT" or "ORB"

# dataset processing 
depth_time, depth_file, rgb_time, rgb_file, gt_time, t_gt, q_gt = dataset_processing()

# initialization
T_est = np.zeros((len(depth_time), 4, 4))
T_prev = np.array([[0.9637 , -0.1439 , 0.2246 , -0.0730],      #initial seed
                   [-0.2666 , -0.5479 , 0.7928 , -0.4169],
                   [0.00901 , -0.8240 , -0.5664 , 1.5916],
                   [0.0 , 0.0 , 0.0 , 1.0]])
T_est[0,:,:] = copy.deepcopy(T_prev)
M_mean_size = np.zeros(len(depth_time))

# max model size
model_size = 5000

# storing corresponding ground truth values
t_gt_new = np.zeros((len(depth_time), 3))
q_gt_new = np.zeros((len(depth_time), 4))

start_time = time.time()

for i in tqdm(range(len(depth_time))):

    # getting matching rgb and ground truth indices
    idx = img_idx(depth_time[i])
    idx2 = img_idx2(depth_time[i])

    # reading images
    depth_I = Image.open((path + depth_file[i]) )
    rgb_I = Image.open(path + rgb_file[idx])

    # storing matching ground truth values
    t_gt_new[i, :] = t_gt[:, idx2]
    q_gt_new[i, :] = q_gt[:, idx2]

    # building feature set
    if feature_detector == "GFTT":
        D_mean, D_cov = get_data_gftt(rgb_I, depth_I)
    elif  feature_detector == "SIFT":
        D_mean, D_cov = get_data_sift(rgb_I, depth_I)
    elif  feature_detector == "ORB":
        D_mean, D_cov = get_data_orb(rgb_I, depth_I)

    # Initializing Model
    if(i == 0):
        M_mean = (T_prev @ D_mean.T).T
        M_cov = T_prev[:3, :3] @ D_cov @ T_prev[:3, :3].T
        M_mean_size[i] = len(M_mean)
        continue

    # registration between data and model using ICP
    # estimating transformation matrix
    correction = icp(D_mean, D_cov, M_mean, M_cov, T_prev)
    T_curr = correction @ T_prev
    T_est[i, :, :] = copy.deepcopy(T_curr)
    T_prev = copy.deepcopy(T_curr)

    # creating dataset D_prime in fixed frame
    Dp_mean = (T_curr @ D_mean.T).T
    Dp_cov =  T_curr[:3, :3] @ D_cov @ T_curr[:3, :3].T

    # Data Association 
    idx_D, idx_M, dist_arr = association_idx(Dp_mean, Dp_cov, M_mean, M_cov, 10.0)

    # Update Associated Features
    if len(idx_D) != 0:
        D_assoc_mean = Dp_mean[idx_D, :]
        D_assoc_cov = Dp_cov[idx_D, :, :]
        M_assoc_mean = M_mean[idx_M, :]
        M_assoc_cov = M_cov[idx_M, :, :]

        # Kalman Filter Update
        M_upd_mean, M_upd_cov = Kalman_Update(D_assoc_mean, D_assoc_cov, M_assoc_mean, M_assoc_cov)
        M_mean[idx_M, :] = M_upd_mean
        M_cov[idx_M, :, :] = M_upd_cov


    if(len(idx_D) < D_mean.shape[0]):
        if len(idx_D) == 0:
            # all new features
            new_feature_idx = np.arange(D_mean.shape[0])    
        else:
            # add non-associated features to model M as new features
            new_feature_idx = np.array(list(set(np.arange(D_mean.shape[0])) - set(idx_D)))

        M_mean = np.append(M_mean, Dp_mean[new_feature_idx, :], axis=0)
        M_cov = np.append(M_cov, Dp_cov[new_feature_idx, :, :], axis=0)
     
    # pruning model
    if(M_mean.shape[0] > model_size):
        diff = M_mean.shape[0] - model_size
        M_mean = np.delete(M_mean, np.s_[:diff], 0)
        M_cov = np.delete(M_cov, np.s_[:diff], 0)

    M_mean_size[i] = len(M_mean)

end_time = time.time()

print("Start Time = ", start_time)
print("End Time = ", end_time)
print("Time Taken (hr) = ", (end_time - start_time)/3600)

save_time = str(time.time())
print("Save Time = ", save_time)

### Plotting Results 

In [ ]:
""" Saving Data """
save_path = path + 'Results/'
np.save(save_path + 'T_est_' + save_time + '.npy', T_est)
np.save(save_path + 't_gt_' + save_time + '.npy', t_gt_new)
np.save(save_path + 'q_gt_' + save_time + '.npy', q_gt_new)
np.save(save_path + 'timestamps_' + save_time + '.npy', np.arange(len(depth_time)))
np.save(save_path + 'M_mean_' + save_time + '.npy', M_mean)
np.save(save_path + 'M_mean_size_' + save_time + '.npy', M_mean_size)

In [ ]:
def quat_to_euler(q):
    """
    Function that converts quaternions to Euler Angles

    Parameters
    ----------
        q - quaternion [x,y,z,w]

    Returns
    -------
        phi, theta, psi - roll, pitch, yaw angles
        
    """

    phi = math.atan2(2*(q[3]*q[0]+q[1]*q[2]), \
            1 - 2*(q[0]**2 + q[1]**2))
    theta = math.asin(2*(q[3]*q[1] - q[2]*q[0]))
    psi = math.atan2(2*(q[3]*q[2]+q[0]*q[1]), \
            1 - 2*(q[1]**2 + q[2]**2))
    
    return np.array([phi, theta, psi])

In [ ]:
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


def plot_results(T_est, t_gt, q_gt, time_steps, M_mean, M_mean_size, full_model,  path, save_time):
    """ Function responsible for plotting """

    print("Plotting...")

    r_est, p_est, y_est = [], [], []
    r_gt, p_gt, y_gt = [], [], []
    tx_est, ty_est, tz_est = [], [], []
    tx_gt, ty_gt, tz_gt = [], [], []

    
    for i in range(len(time_steps)):
        #### GET Rotation data ####
        r, p, y = quat_to_euler(q_gt[i])
        r_gt.append(r)
        p_gt.append(p)
        y_gt.append(y)
        q = R.from_matrix(T_est[i][:3, :3]).as_quat()
        r, p, y = quat_to_euler(q)
        r_est.append(r)
        p_est.append(p)
        y_est.append(y)

        #### GET Translation data ####
        tx, ty, tz = t_gt[i][0], t_gt[i][1], t_gt[i][2]
        tx_gt.append(tx)
        ty_gt.append(ty)
        tz_gt.append(tz)
        tx, ty, tz = T_est[i][0,3], T_est[i][1,3], T_est[i][2,3]
        tx_est.append(tx)
        ty_est.append(ty)
        tz_est.append(tz)

    save_path = path + 'Plots/'
    
    #Plot Rotations
    fig1, ax1 = plt.subplots()
    ax1.plot(time_steps, r_gt, 'r--', markersize=1.5, label='Ground Truth')
    ax1.plot(time_steps, r_est, 'r-', markersize=2, label='Estimate')
    ax1.set_xlabel('Time (secs)')
    ax1.set_ylabel('Roll (rad)')
    ax1.set_title('Roll - Ground truth vs Estimated')
    ax1.legend()
    plt.savefig(save_path + 'Roll/' + save_time + '.png')
    # ax1.legend()

    fig2, ax2 = plt.subplots()
    ax2.plot(time_steps, p_gt, 'g--', markersize=1.5, label='Ground Truth')
    ax2.plot(time_steps, p_est, 'g-', markersize=2, label='Estimate')
    ax2.set_xlabel('Time (secs)')
    ax2.set_ylabel('Pitch (rad)')
    ax2.set_title('Pitch - Ground truth vs Estimated')
    ax2.legend()
    plt.savefig(save_path + 'Pitch/' + save_time + '.png')
    # ax2.legend()

    fig3, ax3 = plt.subplots()
    ax3.plot(time_steps, y_gt, 'b--', markersize=1.5, label='Ground Truth')
    ax3.plot(time_steps, y_est, 'b-', markersize=2, label='Estimate')
    ax3.set_xlabel('Time (secs)')
    ax3.set_ylabel('Yaw (rad)')
    ax3.set_title('Yaw - Ground truth vs Estimated')
    ax3.legend()
    plt.savefig(save_path + 'Yaw/' + save_time + '.png')
    # ax3.legend()

    #Plot Translation
    fig4, ax4 = plt.subplots()
    ax4.plot(time_steps, tx_gt, 'r--', markersize=1.5, label='Ground Truth')
    ax4.plot(time_steps, tx_est, 'r-', markersize=2, label='Estimate')
    ax4.set_xlabel('Time (secs)')
    ax4.set_ylabel('Translation along x-axis (m)')
    ax4.set_title('Translation along x-axis - Ground truth vs Estimated')
    ax4.legend()
    plt.savefig(save_path + 'Tx/' + save_time + '.png')
    # ax4.legend()

    fig5, ax5 = plt.subplots()
    ax5.plot(time_steps, ty_gt, 'g--', markersize=1.5, label='Ground Truth')
    ax5.plot(time_steps, ty_est, 'g-', markersize=2, label='Estimate')
    ax5.set_xlabel('Time (secs)')
    ax5.set_ylabel('Translation along y-axis (m)')
    ax5.legend()
    ax5.set_title('Translation along y-axis - Ground truth vs Estimated')
    plt.savefig(save_path + 'Ty/' + save_time + '.png')
    # ax5.legend()

    fig6, ax6 = plt.subplots()
    ax6.plot(time_steps, tz_gt, 'b--', markersize=1.5, label='Ground Truth')
    ax6.plot(time_steps, tz_est, 'b-', markersize=2, label='Estimate')
    ax6.set_xlabel('Time (secs)')
    ax6.set_ylabel('Translation along z-axis (m)')
    ax6.legend()
    ax6.set_title('Translation along z-axis - Ground truth vs Estimated')
    plt.savefig(save_path + 'Tz/' + save_time + '.png')
    # ax6.legend()

    fig7, ax7 = plt.subplots()
    ax7.plot(time_steps, M_mean_size, 'b--', markersize=1.5, label='Model Size')
    ax7.set_xlabel('Time (secs)')
    ax7.set_ylabel('Model size (num of features)')
    ax7.legend()
    ax7.set_title('Model Size')
    plt.savefig(save_path + 'M_mean_size/' + save_time + '.png')

    #Plot Model
    fig = plt.figure(8)
    axs = fig.gca(projection='3d')
    axs.plot3D(tx_est, ty_est, tz_est, 'k-', markersize=1)
    axs.plot3D([tx_est[0]], [ty_est[0]], [tz_est[0]], 'ro', markersize=5)
    axs.plot3D([tx_est[-1]], [ty_est[-1]], [tz_est[-1]], 'go', markersize=5)
    axs.set_xlabel('x-axis (m)')
    axs.set_ylabel('y-axis (m)')
    axs.set_zlabel('z-axis (m)')
    axs.legend()
    axs.set_title('Final Model Set')
    plt.savefig(save_path + 'Model/' + save_time + '.png')

    plt.show()
    


In [ ]:
""" Plotting Loaded Data """

save_path = path + 'Results/'
#save_time = ''
T_est = np.load(save_path + 'T_est_' + save_time + '.npy')
t_gt = np.load(save_path + 't_gt_' + save_time + '.npy')
q_gt = np.load(save_path + 'q_gt_' + save_time + '.npy')
depth_time = np.load(save_path + 'timestamps_' + save_time + '.npy')
M_mean = np.load(save_path + 'M_mean_' + save_time + '.npy')
M_mean_size = np.load(save_path + 'M_mean_size_' + save_time + '.npy')
full_model = np.load(save_path + 'Full_model_' + save_time + '.npy')

plot_results(T_est, t_gt, q_gt, depth_time, M_mean, M_mean_size, full_model, path, save_time)


### Evaluation 

In [ ]:
# Evaluation - Absolute Trajectory Error (ATE)
def absolute_traj_error(t_gt, T_est):
    """ 
    Function that evaluates the estimted trajectory with ground truth
    
    Parameters
    ----------
        t_gt  - ground truth trajectory
        T_est - estimated trajectory
    """

    t_est = T_est[:,:3,3]
    t_err = np.linalg.norm((t_gt - t_est), axis=1) 
    t_err_rmse = np.sqrt(np.mean(t_err**2))
    t_err_mean = np.mean(t_err)
    t_err_median = np.median(t_err)
    t_err_std = np.std(t_err)
    t_err_min = np.min(t_err)
    t_err_max = np.max(t_err)

    print("RMS Error: ", t_err_rmse)
    print("Mean Error: ", t_err_mean)
    print("Median of Error: ", t_err_median)
    print("Standard Deviation of Error: ", t_err_std)
    print("Minimum Error: ", t_err_min)
    print("Maximum Error: ", t_err_max)

absolute_traj_error(t_gt, T_est)